In [1]:
pwd

'/home/ankith/Portfolio/kaggle/jigsaw/notebooks'

In [2]:
import random

import pandas as pd
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [3]:
model = ModelFactory.create(
    ModelPlatformType.GEMINI, ModelType.GEMINI_2_5_FLASH
)
agent = ChatAgent(model=model)

## Rules Generation

its proven there are more than 2 rules in the public & private test set - having more rules will help us generalize better


In [4]:
df = pd.read_csv("../dataset/train.csv")

In [40]:
df

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1
...,...,...,...,...,...,...,...,...,...
2024,2024,Please edit your post so it is readable. These...,No legal advice: Do not offer or request legal...,relationships,"I'm not ok with this in anyway, and think you ...",See a lawyer under the guise that you want thi...,"This is just untrue. OP is 13, not an adult -...",Why should I care about all the bicyclists I i...,1
2025,2025,"Yes, and in a right to work state they can eve...",No legal advice: Do not offer or request legal...,legaladvice,Move as much of your assets as you can offshor...,We have great consumer protection laws. There'...,"LPT piratebay, transmission, vpn. Get the musi...","It's not so much that I killed them, it's that...",0
2026,2026,**HD** Streams: |ENG **HD**[ Watch here..PC & ...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,stitopdisca1987.tumblr.com - sex Take girl for...,this girl get sex going to to old http://mrk....,NO ADS | NO ADS | NO ADS\n\nWe show all SOCCER...,[So great! Thanks for sharing.](http://www.che...,1
2027,2027,No. Not when doing so obviously presents a saf...,No legal advice: Do not offer or request legal...,politics,SHE ISNT A BIRTHING CHAMBER BUT EQUALLY THE BA...,"Jail? What are you, ten years old? If they pro...",Who cares about that when I can keep raping in...,send me a private message; I may be able to he...,1


In [5]:
df.sample()

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
1265,1265,ヽ༼ ຈل͜ຈ༽ ﾉ Raise Them!\n\n ^^Dongers ^^Raised:...,"No Advertising: Spam, referral links, unsolici...",conspiracy,One guy have sex with all best girls there you...,Flip Knife | Safari Mesh (Field Tested)\n\nI h...,Oh that's interesting. You should check out my...,[WEB ONLY] [AdBlock OK] SD - ENGLISH [Stream #...,0


In [5]:
rules = df["rule"].unique().tolist()
rule_1_df = df[df["rule"] == rules[0]]
rule_2_df = df[df["rule"] == rules[1]]
rule_1_violate = rule_1_df[rule_1_df["rule_violation"] == 1]
rule_2_violate = rule_2_df[rule_2_df["rule_violation"] == 1]
rule_1_non_violate = rule_1_df[rule_1_df["rule_violation"] == 0]
rule_2_non_violate = rule_2_df[rule_2_df["rule_violation"] == 0]

In [80]:
def sample_prompt_violation():
    prompt = """You are a Reddit Rule Generator. Your task is to create a rule that the provided comments violate, you will be given examples of comments and its violated rule.
<Examples>
Comments: [{}, {}, {}]
Rule Violation: True
Rule: {}

Comments: [{}, {}, {}]
Rule Violation: True
Rule: {}
</Examples>

Generate a robust yet different reddit rule that is being violated by the following comments.
Comments:[{}, {}, {}]
Rule Violation: True
Rule:
"""
    rule = random.choice(["rule1", "rule2"])
    rule_1_sample_size = 3
    rule_2_sample_size = 3
    rule1_body = pd.DataFrame(rule_1_violate["body"].unique())
    rule2_body = pd.DataFrame(rule_2_violate["body"].unique())

    if rule == "rule1":
        rule_1_sample_size += 3

        rule1_oversamples = rule1_body.sample(rule_1_sample_size)

        rule2_samples = rule2_body.sample(rule_2_sample_size)

        rule1_samples = rule1_oversamples[:3]
        comments_args = rule1_oversamples[3:][0].tolist()

    else:
        rule_2_sample_size += 3

        rule1_samples = rule1_body.sample(rule_1_sample_size)

        rule2_oversamples = rule2_body.sample(rule_2_sample_size)

        rule2_samples = rule2_oversamples[:3]
        comments_args = rule2_oversamples[3:][0].tolist()

    rule_1_args = (*rule1_samples[0].tolist(), rules[0])
    rule_2_args = (*rule2_samples[0].tolist(), rules[1])

    prompt_args = (*rule_1_args, *rule_2_args, *comments_args)

    return prompt.format(*prompt_args)

In [84]:
def sample_prompt_non_violation():
    prompt = """You are a Reddit Rule Generator. Your task is to create a rule that the provided comments do not violate, you will be given examples of comments and its non violated rule.
<Examples>
Comments: [{}, {}, {}]
Rule Violation: False
Rule: {}

Comments: [{}, {}, {}]
Rule Violation: False
Rule: {}
</Examples>

Generate a robust yet different reddit rule that is not being violated by the following comments.
Comments:[{}, {}, {}]
Rule Violation: False
Rule:
"""
    rule = random.choice(["rule1", "rule2"])
    rule_1_sample_size = 3
    rule_2_sample_size = 3
    rule1_body = pd.DataFrame(rule_1_non_violate["body"].unique())
    rule2_body = pd.DataFrame(rule_2_non_violate["body"].unique())

    if rule == "rule1":
        rule_1_sample_size += 3

        rule1_oversamples = rule1_body.sample(rule_1_sample_size)

        rule2_samples = rule2_body.sample(rule_2_sample_size)

        rule1_samples = rule1_oversamples[:3]
        comments_args = rule1_oversamples[3:][0].tolist()

    else:
        rule_2_sample_size += 3

        rule1_samples = rule1_body.sample(rule_1_sample_size)

        rule2_oversamples = rule2_body.sample(rule_2_sample_size)

        rule2_samples = rule2_oversamples[:3]
        comments_args = rule2_oversamples[3:][0].tolist()

    rule_1_args = (*rule1_samples[0].tolist(), rules[0])
    rule_2_args = (*rule2_samples[0].tolist(), rules[1])

    prompt_args = (*rule_1_args, *rule_2_args, *comments_args)

    return prompt.format(*prompt_args)

In [81]:
print(sample_prompt_violation())

You are a Reddit Rule Generator. Your task is to create a rule that the provided comments violate, you will be given examples of comments and its violated rule.
<Examples>
Comments: [FindSexToday . com - free girls for sex worldwide 26h0nowzvkoV7!, The two greatest swords of all time have already been created. For a mere $3000 they can be yours!

http://filmswords.com/conan/conan-sword-atlantean-anniversary.htm

http://filmswords.com/conan/conan-sword-master-fathers-anniversarys.htm, FindSexToday . com - free girls for sex worldwide 26Yg93ZelUJu2!]
Rule Violation: True
Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.

Comments: [I really hope one of those children grows up and sues your mum for what she did to them., As soon as Sanders formally endorses secretary Clinton, he's LEGALLY OBLIGATED to provide her with campaign with financial aid! All of reddit's money is going straight to Clinton, no matter what.

So you don't ha

In [85]:
print(sample_prompt_non_violation())

You are a Reddit Rule Generator. Your task is to create a rule that the provided comments do not violate, you will be given examples of comments and its non violated rule.
<Examples>
Comments: [SD Stream DDen - [TV3 Sport 2 Denmark](http://soccerstreamsarena.com/fc-midtjylland-vs-esbjerg-fb-live-danish-superligaen-09-sep-2016/) 

Mobile : Compatible With Android Puffin Browser, must be watch movie https://sites.google.com/site/thereturn2016hdfullmovie/watch-car-dogs-2014-hd-quality-streaming, Here is my discussion.review video of Re: Zero Episode 14 "The Sickness called Despair"! Tune in and make sure to SUBSCRIBE!
https://www.youtube.com/watch?v=LOakyvdPfx0]
Rule Violation: False
Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.

Comments: [Help me out with the details and I'll place an order. I want salt that won't melt with water but will rust the fuck out of their cars. PM pls, I know for sure the women beating thing is ex

Generate 5 rules for violation and 5 rules for non violation


In [ ]:
violation_rules = []
non_violation_rules = []

for iter in range(10):
    if iter < 5:
        print("Generating violation rules")
        prompt = sample_prompt_violation()
        response = agent.step(prompt)
        print(response.msg.content if response.msg else "")
        violation_rules.append(response.msg.content if response.msg else "")
        agent.reset()
    else:
        print("Generating non violation rules")
        prompt = sample_prompt_non_violation()
        response = agent.step(prompt)
        print(response.msg.content if response.msg else "")
        non_violation_rules.append(
            response.msg.content if response.msg else ""
        )
        agent.reset()

Generating violation rules


/home/ankith/Portfolio/kaggle/jigsaw/.venv/lib/python3.10/site-packages/camel/memories/blocks/chat_history_block.py:73: UserWarning: The `ChatHistoryMemory` is empty.
  warnings.warn("The `ChatHistoryMemory` is empty.")


Rule: No Disruptive Behavior: Personal attacks, harassment, direct solicitations for personal aid, or comments that otherwise detract from constructive discussion are not allowed.
Generating violation rules
No promoting violence or harassment: Do not encourage or advocate for violence, physical harm, or harassment against individuals or groups.
Generating violation rules
Rule: No Solicitation of Transactions or Commercial Activity: Do not use the subreddit to buy, sell, trade, or solicit specific goods, services, accounts, digital items, gift codes, or financial products. This includes any direct offers or requests for monetary or value-based exchanges.
Generating violation rules
Rule: No Combative Discourse: Do not engage in aggressive, hostile, or overly confrontational argumentation. Debates should remain respectful and constructive.
Generating violation rules
No legal opinions or advice for individual circumstances.
Generating violation rules
Rule: **No Doxxing or Sharing Personal 

In [92]:
rules = violation_rules + non_violation_rules

In [96]:
import json

with open("../dataset/synthetic/rules.json", "w") as file:
    json.dump(rules, file, indent=4)

Use generated rules to create a synthetic dataset of rules -> violating and non-violating comments


In [6]:
def sample_prompt_violation_comments(rule):
    prompt = """You are a Reddit Comment Generator. Your task is to create a set of comments that violates a given rule, you will be given examples of comments that violate a certain set of rules.
<Examples>
Rule Violation: True
Rule: {}
Comments: [{}, {}, {}]

Rule Violation: True
Rule: {}
Comments: [{}, {}, {}]
</Examples>

Generate a list of 10 diverse yet robust reddit comments that violate the following rule. Use slang, typos or personas whenever appropriate.
Rule Violation: True
Rule: {}
Comments:
"""
    rule_1_sample_size = 3
    rule_2_sample_size = 3
    rule1_body = pd.DataFrame(rule_1_violate["body"].unique())
    rule2_body = pd.DataFrame(rule_2_violate["body"].unique())

    rule1_samples = rule1_body.sample(rule_1_sample_size)

    rule2_samples = rule2_body.sample(rule_2_sample_size)

    rule_1_args = (rules[0], *rule1_samples[0].tolist())
    rule_2_args = (rules[1], *rule2_samples[0].tolist())

    prompt_args = (*rule_1_args, *rule_2_args, rule)

    return prompt.format(*prompt_args)

In [7]:
def sample_prompt_non_violating_comments(rule):
    prompt = """You are a Reddit Comment Generator. Your task is to create a set of comments that do not violate a given rule, you will be given examples of comments that don't violate a certain set of rules.
<Examples>
Rule Violation: False
Rule: {}
Comments: [{}, {}, {}]

Rule Violation: False
Rule: {}
Comments: [{}, {}, {}]
</Examples>

Generate a list of 10 diverse yet robust reddit comments that do not violate the following rule. Use slang, typos or personas whenever appropriate.
Rule Violation: False
Rule: {}
Comments:
"""
    rule_1_sample_size = 3
    rule_2_sample_size = 3
    rule1_body = pd.DataFrame(rule_1_non_violate["body"].unique())
    rule2_body = pd.DataFrame(rule_2_non_violate["body"].unique())

    rule1_samples = rule1_body.sample(rule_1_sample_size)

    rule2_samples = rule2_body.sample(rule_2_sample_size)

    rule_1_args = (rules[0], *rule1_samples[0].tolist())
    rule_2_args = (rules[1], *rule2_samples[0].tolist())

    prompt_args = (*rule_1_args, *rule_2_args, rule)

    return prompt.format(*prompt_args)

## Generate Data


In [8]:
import json
from typing import List

from pydantic import BaseModel


class Comments(BaseModel):
    items: List[str]

In [9]:
with open("../dataset/synthetic/rules.json", "r") as file:
    rules_json = json.load(file)

In [13]:
rules_df = pd.DataFrame(rules_json, columns=["rule"])

In [ ]:
with open("../dataset/synthetic/comments.json", "r") as file:
    comments_dict = json.load(file)
comments_dict

[]

In [24]:
import time

attribute_max = 20
for _ in range(250):
    if attribute_max > 0:
        time.sleep(1)
        rule = rules_df.sample()["rule"].tolist()[0]
        print("")
        print("Generating violation comments")
        prompt = sample_prompt_violation_comments(rule)
        try:
            response = agent.step(prompt, response_format=Comments)
        except AttributeError:
            attribute_max -= 1
            continue
        comment_list: List[str] = (
            response.msg.parsed.items if response.msg else ""
        )  # type: ignore

        with open("../dataset/synthetic/comments.json", "r") as file:
            comments_dict = json.load(file)

        for comment in comment_list:
            synthetic_sample_dict = {
                "rule": rule,
                "body": comment,
                "rule_violation": 1,
            }
            comments_dict.append(synthetic_sample_dict)

        with open("../dataset/synthetic/comments.json", "w") as file:
            json.dump(comments_dict, file, indent=4)

        agent.reset()
        time.sleep(1)
        print("")
        print("Generating non violation comments")
        prompt = sample_prompt_non_violating_comments(rule)
        try:
            response = agent.step(prompt, response_format=Comments)
        except AttributeError:
            attribute_max -= 1
            continue
        comment_list: List[str] = (
            response.msg.parsed.items if response.msg else ""
        )  # type: ignore

        with open("../dataset/synthetic/comments.json", "r") as file:
            comments_dict = json.load(file)

        for comment in comment_list:
            synthetic_sample_dict = {
                "rule": rule,
                "body": comment,
                "rule_violation": 0,
            }
            comments_dict.append(synthetic_sample_dict)

        with open("../dataset/synthetic/comments.json", "w") as file:
            json.dump(comments_dict, file, indent=4)
        agent.reset()


Generating violation comments
2025-09-15 09:35:17,521 - camel.models.model_manager - ERROR - Error processing with model: <camel.models.gemini_model.GeminiModel object at 0x7f67000d8700>
2025-09-15 09:35:17,524 - camel.camel.agents.chat_agent - WARNING - Rate limit hit (attempt 1/3). Retrying in 0.8s
2025-09-15 09:35:21,961 - camel.models.model_manager - ERROR - Error processing with model: <camel.models.gemini_model.GeminiModel object at 0x7f67000d8700>
2025-09-15 09:35:21,963 - camel.camel.agents.chat_agent - WARNING - Rate limit hit (attempt 2/3). Retrying in 1.0s
2025-09-15 09:35:26,050 - camel.models.model_manager - ERROR - Error processing with model: <camel.models.gemini_model.GeminiModel object at 0x7f67000d8700>
2025-09-15 09:35:26,051 - camel.camel.agents.chat_agent - ERROR - Rate limit exhausted after 3 attempts


ModelProcessingError: Unable to process messages: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'}, 'quotaValue': '250'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '31s'}]}}]